# Augmentation Methodology
The following describes the methodology used to augment the driver information, given that we are going to collapse the data into a frame with single rows per driver. Since we will be inherently losing information by collapsing all of the rows, we will need to supplement the dataset with new columns.

## Temporal Data
Most of the temporal information will be lost when we collapse the rows, so we will need additional columns that summarize that info:

- Start date (first load)
- End date (most recent load)
- Longevity (how long the driver has been employed, according to this dataset)
- Trip Frequency (standard deviation or some measure of variance between trips)
- Load Delta (some standard measure of how many days between each load)
- Preferred weekday (the mode of the `weekday` column
- Normal weekday (the mean or median of the `weekday` column
- Weekday distribution (are the weekdays consistent?)

## Location Data

- Roaming (does the driver have more than one reported `home_base_city` or `home_base_state`?
- Preferred Trip Distance (Possibly use `interested_in_dryage` and `carrier_trucks` to determine a measure of how far the driver prefers to drive)

## Boolean Features

- Indecision (did this person apply to become a driver more than once?)

---

# Transformation Methodology

## Converting categoricals to boolean

The following columns have only two unique values, which are easily turned into boolean features:

- `dim_carrier_type` (corresponds to "is this a self-owned driver?", **0: Fleet, 1: Owner Operator**)
- `interested_in_drayage` (corresponds to "is this driver willing/able to complete drayage shipments?", **0: not specified, 1: yes**)
- `port_qualified` (corresponds to "is this driver qualified to handle port shipments?", **0: no, 1: yes**)
- `signup_source` (corresponds to "did this driver sign up on a mobile device?", **0: Other, 1: Mobile**)
- `driver_with_twic` (corresponds to "does this driver have TWIC insurance?" **0: no, 1: yes**)

## One Hot Encoding Categoricals

The following columns have a limited number of unique categorical values that would work well with one-hot encoding

- `carrier_trucks` (16 unique arrangements of trucks)
- `weekday` (**possible** -- pro: maintain all information for day of week, con: 7 extra columns, most of which will be sparse)
- `dim_preferred_lanes` (**possible** -- 53 unique values -- pro: this might actually mean something, con: it might mean absolutely nothing and will double the number of columns)

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import seaborn as sn
import datetime
import random
from datetime import date
from pprint import pprint
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import statistics 
from statistics import mode

In [2]:
def aggregate_rows_by_driver_id(dataframe):
    '''
    Returns a new dataframe indexed by Driver ID using an optimal set of aggregations per column.
    This method will return a dataframe that only contains the columns listed below.
    Note: duplicate columns are inherently pruned (simply by not adding them below), so it is 
        safe to pass in the full dataframe and expect a pruned version in return. 
        Also note that the `id_driver` will now be the index of the dataframe and NOT its own column.
    '''
    minimum = 'min'
    maximum = 'max'
    median = 'median'
    random_mode = lambda x: random.choice(pd.Series.mode(x if isinstance(x, list) else list(x)))

    aggregation = {
        'dt': maximum,
        'weekday': random_mode,
        'id_carrier_number': random_mode,
        'dim_carrier_type': random_mode,
        'dim_carrier_company_name': mode,
        'home_base_city': mode,
        'home_base_state': mode,
        'carrier_trucks': mode,
        'num_trucks': maximum,
        'interested_in_drayage': random_mode,
        'port_qualified': random_mode,
        'signup_source': random_mode,
        'ts_signup': maximum,
        'ts_first_approved': minimum,
        'days_signup_to_approval': maximum,
        'driver_with_twic': mode,
        'dim_preferred_lanes': mode,
        'first_load_date': minimum,
        'most_recent_load_date': maximum,
        'loads': random_mode,
        'marketplace_loads_otr': maximum,
        'marketplace_loads_atlas': maximum,
        'marketplace_loads': maximum,
        'brokerage_loads_otr': maximum,
        'brokerage_loads_atlas': maximum,
        'brokerage_loads': maximum,
        'total_loads': maximum
    }

    return dataframe.groupby(['id_driver']).agg(aggregation)

In [3]:
df = pd.read_csv('training_dataset_V3.csv')
df.shape

(83414, 31)

In [4]:
df2 = aggregate_rows_by_driver_id(df)

In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5291 entries, 20 to 38125
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   dt                        5291 non-null   object 
 1   weekday                   5291 non-null   object 
 2   id_carrier_number         5291 non-null   object 
 3   dim_carrier_type          5291 non-null   object 
 4   dim_carrier_company_name  5284 non-null   object 
 5   home_base_city            5279 non-null   object 
 6   home_base_state           5279 non-null   object 
 7   carrier_trucks            5291 non-null   object 
 8   num_trucks                5249 non-null   float64
 9   interested_in_drayage     5291 non-null   object 
 10  port_qualified            5291 non-null   object 
 11  signup_source             5291 non-null   object 
 12  ts_signup                 5291 non-null   object 
 13  ts_first_approved         3962 non-null   object 
 14  days_s

In [6]:
df2.nunique()

dt                          1418
weekday                        7
id_carrier_number           2572
dim_carrier_type               2
dim_carrier_company_name    2491
home_base_city               520
home_base_state               40
carrier_trucks                16
num_trucks                    30
interested_in_drayage          2
port_qualified                 2
signup_source                  2
ts_signup                   2573
ts_first_approved           1880
days_signup_to_approval      674
driver_with_twic               2
dim_preferred_lanes           53
first_load_date             1458
most_recent_load_date       1418
loads                         13
marketplace_loads_otr        108
marketplace_loads_atlas      196
marketplace_loads            228
brokerage_loads_otr          216
brokerage_loads_atlas        101
brokerage_loads              241
total_loads                  331
dtype: int64